<a href="https://colab.research.google.com/github/sergiokapone/DataScience/blob/main/Hw7/Hw7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання
Візьміть датасет [movielens](https://surprise.readthedocs.io/en/stable/dataset.html) і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими [алгоритмами](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html) розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.


In [1]:
!pip install scikit-surprise

In [2]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from hyperopt import hp, fmin, tpe
import pandas as pd


In [3]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

In [4]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
train_set, test_set = train_test_split(data, test_size=0.25)

Для вибору параметрів скористаємось бібліотекою `hyperopt`. Об'єкт `hp`   

In [5]:
param_svd = {
    'n_factors': hp.choice('n_factors', [50, 100, 150]),
    'n_epochs': hp.choice('n_epochs', [10, 20, 30]),
    'lr_all': hp.uniform('lr_all', 0.001, 0.1),
    'reg_all': hp.uniform('reg_all', 0.01, 0.2),
    'random_state': 42
}

param_nmf = {
        'n_factors': hp.choice('n_factors', [5, 10, 15, 20, 25]),
        'n_epochs': hp.choice('n_epochs', [10, 20, 30, 40, 50]),
        'biased': hp.choice('biased', [True, False]),
        'reg_pu': hp.uniform('reg_pu', 0.001, 0.1),
        'reg_qi': hp.uniform('reg_qi', 0.001, 0.1),
        'lr_bu': hp.uniform('lr_bu', 0.001, 0.1),
        'lr_bi': hp.uniform('lr_bi', 0.001, 0.1),
        'random_state': 42
    }

In [6]:
params = [param_svd,
          param_svd,
          param_nmf]
models = [SVD,
          SVDpp,
          NMF]

https://github.com/hyperopt/hyperopt/wiki/FMin

In [7]:

def objective(params, func):
    model = func(**params)
    results = cross_validate(model, data, measures=['RMSE'], cv=5, verbose=False)
    return results['test_rmse'].mean()

best_params = []
for param, func in zip(params, models):
    print('-------------------------')
    print(f'Srart: {func.__name__}')
    print('-------------------------')

    best = fmin(fn=lambda params: objective(params, func), space=param, algo=tpe.suggest, max_evals=5)
    best_params.append(best)


-------------------------
Srart: SVD
-------------------------
100%|██████████| 5/5 [00:47<00:00,  9.43s/trial, best loss: 0.9352651231913093]
-------------------------
Srart: SVDpp
-------------------------
100%|██████████| 5/5 [53:18<00:00, 639.78s/trial, best loss: 0.9177533856239064]
-------------------------
Srart: NMF
-------------------------
100%|██████████| 5/5 [00:32<00:00,  6.43s/trial, best loss: 1.3704223355350291]


In [8]:
best_params

[{'lr_all': 0.004281257232894452,
  'n_epochs': 2,
  'n_factors': 2,
  'reg_all': 0.12356955855587302},
 {'lr_all': 0.023100152820063505,
  'n_epochs': 0,
  'n_factors': 1,
  'reg_all': 0.05442389272055986},
 {'biased': 1,
  'lr_bi': 0.031422342908873815,
  'lr_bu': 0.08974440446461929,
  'n_epochs': 0,
  'n_factors': 3,
  'reg_pu': 0.07098843083429891,
  'reg_qi': 0.004282008442886127}]

# Висновки

На основі результатів крос-валідації для різних алгоритмів матричної факторизації (SVD, SVD++, NMF), можна зробити такі висновки:

1. SVD має середнє значення RMSE приблизно 0.93. Час навчання та тестування відносно низький.

2. SVD++ має значення RMSE приблизно 0.917. Час тестування відносно високий, що може бути недоцільним у великих завданнях.

3. NMF має значення RMSE приблизно 1.37. Час навчання та тестування прийнятний, але середня точність менше, ніж у SVD.

За зазначеними метриками RMSE і MAE, SVD має найкращу точність серед цих алгоритмів. Таким чином, для цього конкретного завдання рекомендується використовувати алгоритм SVD для матричної факторизації.